In [3]:
import pandas as pd

In [4]:
# import csv, specific some columns as strings
df = pd.read_csv("../../data/preprocessed_data/subsampled_preprocessed.csv")
#                  dtype={'Pickup_day':object,
#                         'day_of_week': object,
#                        'Pickup_hour':object,
#                        'Pickup_minute':object,
#                        'Pickup_week':object})

In [5]:
df2 = df.copy()[['Pickup_week', 'Pickup_day', 'Pickup_wday', 'Pickup_hour', 'Pickup_minute', 'Pickup_cell', 'Number_trips']]

In [6]:
df2['Pickup_minute'] = df2['Pickup_minute'].apply(lambda x: x//10 * 10)

In [7]:
df2 = df2.groupby(['Pickup_week', 'Pickup_day', 'Pickup_wday', 'Pickup_hour', 'Pickup_minute', 'Pickup_cell'
                  ]).sum().reset_index()

# Train Model

### Poisson regression

In [8]:
# Poisson regression
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [9]:
formula = 'Number_trips~ Pickup_week + Pickup_day + Pickup_hour + \
                        Pickup_minute+Pickup_cell'

data = df2

In [10]:
pois_glm = smf.glm(formula=formula, data=data, family=sm.families.Poisson()).fit()

MemoryError: 

In [14]:
%reset -s -f

In [82]:
import pickle

In [83]:
with open('../players/model/pois_glm.pkl', 'wb') as handle:
    pickle.dump(pois_glm, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Linear Regression

In [16]:
# Linear Regression
from sklearn.linear_model import LinearRegression

In [17]:
X = df_st.loc[:, ["PickupCell","day_of_week","minute_of_day"]]
y = df_st.loc[:,"frequency"].astype('float')

In [18]:
from sklearn.preprocessing import OneHotEncoder

In [19]:
ohe = OneHotEncoder()

In [20]:
train = ohe.fit_transform(X).toarray()
train

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [90]:
lr = LinearRegression()
lr.fit(train, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [96]:
import pickle

In [97]:
with open('../model/lr.pickle', 'wb') as handle:
    pickle.dump(lr, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [98]:
with open('../model/ohe.pickle', 'wb') as handle:
    pickle.dump(ohe, handle, protocol=pickle.HIGHEST_PROTOCOL)